In [1]:
from video_downloader import YouTubeDownloader

youtube_downloader = YouTubeDownloader()
youtube_downloader.download_video("https://youtu.be/7tZoIDgpUH4?si=Z5NSTnqfLjizuQ6-","test.mp4",resolution="720p")

'YouTube video downloaded successfully'

In [1]:
from text_downloader import BlogDownloader


In [2]:
blog_downloader = BlogDownloader()
text,msg = blog_downloader.download_text("https://www.ndtv.com/offbeat/parle-g-replaces-iconic-girls-image-with-this-instagram-influencers-face-heres-why-4753827")


print(text)

Parle-G Replaces Iconic Girl's Image With This Instagram Influencer's Face, Here's Why
NDTVProfitWorld EditionहिंदीCricketMOVIESFOODLIFESTYLEHEALTHTECHGamesBig BonusSWASTHSHOPPINGLIVE TV LATEST INDIA VIDEO WORLD OPINION CITIES Education OFFBEAT Feature TRENDS SOUTHPEOPLESciencePhotosWEATHERTV ScheduleTrainsParle-G Replaces Iconic Girl's Image With This Instagram Influencer's Face, Here's WhySearchNewsOffbeatParle-G Replaces Iconic Girl's Image With This Instagram Influencer's Face, Here's WhyRead inOther Languages हिंदी में पढ़ें Parle-G Replaces Iconic Girl's Image With This Instagram Influencer's Face, Here's Why  The amusing post was in response to content creator Zervaan J Bunshah's viral video where he put forth a hilarious question to his followers. OffbeatEdited by Ritu SinghUpdated: December 27, 2023 11:39 pm ISTParle-G featured a smiling image of Mr Bunshah on the biscuit wrapper instead of the iconic girlBiscuit manufacturer Parle left internet users surprised after it shared